In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
from transformers import BertTokenizer, RobertaForSequenceClassification
import torch
import tkinter.messagebox as messagebox
# Load the saved capsule-net model
model_path = 'capsule_net new.pth'
model_state_dict = torch.load(model_path)
model = RobertaForSequenceClassification.from_pretrained('roberta-base', state_dict=model_state_dict)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create the tkinter window
window = tk.Tk()

# Load the background image
background_image = Image.open('social.png')
image_width, image_height = background_image.size
background_image_tk = ImageTk.PhotoImage(background_image)

# Set the background image as the window's background
window.configure(background='black')

# Set the size of the window according to the size of the background image
window.geometry(f"{image_width}x{image_height}")

# Create a label widget to display the image
background_label = tk.Label(window, image=background_image_tk, width=image_width, height=image_height)
background_label.place(x=0, y=0)

# Create the input box
input_box = tk.Text(window, height=5, width=50)
input_box.pack()

# Create the output box
output_box = tk.Label(window, text='')
output_box.pack()

# Define the function that will be called when the button is clicked
# Define the function that will be called when the button is clicked
def predict():
    # Get the text from the input box
    user_input = input_box.get('1.0', 'end-1c')

    # Tokenize the user input
    tokenized_input = tokenizer.tokenize(user_input)
    input_ids = tokenizer.convert_tokens_to_ids(tokenized_input)
    input_ids = torch.tensor([input_ids])
    attention_masks = torch.where(input_ids != 0, torch.tensor(1), torch.tensor(0))

    # Make prediction
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_masks)
        logits = outputs[0]
        predictions = torch.argmax(logits, dim=1)

    # Print the prediction
    for prediction in predictions:
        if prediction == 1:
            output_box.config(text='The comment is not toxic.')
        else:
            output_box.config(text='The comment is  toxic.')
            # Check if the input comment is longer than a threshold and display a warning message
            if len(user_input) > 3 and prediction == 0:
                messagebox.showwarning('Warning', 'The comment you entered is removed!')

# Create the button
button = tk.Button(window, text='Predict', command=predict)
button.pack()
# Run the tkinter mainloop
window.resizable(0, 0)

# Run the tkinter mainloop
window.mainloop()
